<a href="https://colab.research.google.com/github/Carlos1729/DGL/blob/main/Node_Classification_with_DGL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install  dgl -f https://data.dgl.ai/wheels/cu116/repo.html

Looking in links: https://data.dgl.ai/wheels/cu116/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 11.3 MB/s eta 0:00:00


In [3]:
!pip install  dglgo -f https://data.dgl.ai/wheels-test/repo.html

Looking in links: https://data.dgl.ai/wheels-test/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 55.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7028 sha256=60c9e4908433f123f7150f007bd9dfc73b421faa047fe5308aeb4d349bf70c97
  Stored in directory: /root/.cache/pip/wheels/3d/fe/b0/27a9892da57472e538c7452a721a9cf463cc

In [9]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

One of the most popular and widely adopted tasks on graph data is node classification, where a model needs to predict the ground truth category of each node. Before graph neural networks, many proposed methods are using either connectivity alone (such as DeepWalk or node2vec), or simple combinations of connectivity and the node’s own features. GNNs, by contrast, offers an opportunity to obtain node representations by combining the connectivity and features of a local neighborhood.

Kipf et al., is an example that formulates the node classification problem as a semi-supervised node classification task. With the help of only a small portion of labeled nodes, a graph neural network (GNN) can accurately predict the node category of the others.

This tutorial will show how to build such a GNN for semi-supervised node classification with only a small number of labels on the Cora dataset, a citation network with papers as nodes and citations as edges. The task is to predict the category of a given paper. Each paper node contains a word count vector as its features, normalized so that they sum up to one, as described in Section 5.2 of the paper.

In [10]:
import dgl.data

dataset = dgl.data.CoraGraphDataset()
print('Number of categories:', dataset.num_classes)

Extracting file to /root/.dgl/cora_v2_d697a464
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
Number of categories: 7


In [11]:
g = dataset[0]


In [12]:
print('Node features')
print(g.ndata)
print('Edge features')
print(g.edata)

Node features
{'train_mask': tensor([ True,  True,  True,  ..., False, False, False]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])}
Edge features
{}


In [13]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

# Create the model with given dimensions
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)

In [14]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)
train(g, model)

In epoch 0, loss: 1.946, val acc: 0.072 (best 0.072), test acc: 0.081 (best 0.081)
In epoch 5, loss: 1.892, val acc: 0.450 (best 0.450), test acc: 0.447 (best 0.447)
In epoch 10, loss: 1.805, val acc: 0.500 (best 0.500), test acc: 0.544 (best 0.544)
In epoch 15, loss: 1.694, val acc: 0.642 (best 0.642), test acc: 0.656 (best 0.656)
In epoch 20, loss: 1.557, val acc: 0.684 (best 0.684), test acc: 0.683 (best 0.683)
In epoch 25, loss: 1.400, val acc: 0.716 (best 0.716), test acc: 0.717 (best 0.717)
In epoch 30, loss: 1.229, val acc: 0.738 (best 0.738), test acc: 0.749 (best 0.741)
In epoch 35, loss: 1.053, val acc: 0.750 (best 0.750), test acc: 0.758 (best 0.750)
In epoch 40, loss: 0.883, val acc: 0.756 (best 0.756), test acc: 0.767 (best 0.767)
In epoch 45, loss: 0.728, val acc: 0.766 (best 0.766), test acc: 0.767 (best 0.767)
In epoch 50, loss: 0.593, val acc: 0.770 (best 0.770), test acc: 0.767 (best 0.766)
In epoch 55, loss: 0.481, val acc: 0.772 (best 0.772), test acc: 0.779 (best 0

In [15]:
https://www.youtube.com/watch?v=IMcj0GU5md4

SyntaxError: ignored

In [16]:
https://docs.dgl.ai/en/0.8.x/tutorials/large/L1_large_node_classification.html

SyntaxError: ignored

###Node Classification


https://docs.dgl.al/en/0.9.x/guide/training-node.html

The nodes and edges of a DGLGraph can have several user-defined named features for storing graph-specific properties of the nodes and edges. These features cane accessed via the ndata and edata interface

Double-click (or enter) to edit

A feature is created via tensor assignment, which assigns a feature to each node/edge in the graph. The leading dimension of that tensor must be equal to the number of nodes/edges in the graph.

In [7]:
# The feature tensor is in row-major layout each row-slice
# stores the feature of one node or edge

To classify nodes, a graph neural network performs message passing to utilize the node's own features and its neighboring node and edge features.
For message passing, update_all is a high-level API that merges message generation, message aggregation, and node update in a single call, leaving room for optimization as a whole.
The parameters for update_all are a message function, a reduce function, and an update function
The parameters for update all are a message function, a reduce lnction and an update function.



In [8]:
def update_all_example(graph):
    # Store the result in graph.ndata['ft']
    graph.update_all(fn.u_mul_e('ft', 'a', 'm'), fn.sum('n', 'ft'))

    # Call update function outside of update_all
    final_ft = graph.ndata['ft']

    return final_ft

This call will generate the messages m by multiply are node features ft and edge features a, surn up the messages m to update node features ft, and finally multiply ft by 2 to get the result final ft. After the call, DGL will clean the Intermediate messages m.

DGL provides a few built-in graph convolution modules that can perform one round of message passing. In this guide, we choose dgl.nn.pytorch.SAGEConv (also available in MXNet and Tensorflow), the graph convolution module for GraphSAGE.